In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
import main
import random
import numpy as np
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser

wandb: Currently logged in as: chucooleg (use `wandb login --relogin` to force relogin)


In [2]:
# arguments
parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

args.data_path = '../Raw_Datasets/3Attr-4Vals-2hat23Train-2hat12Val.json'
args.config_path = 'config.json'
args.mode = 'train'
args.generate_full_matrix = True

In [3]:
game_data_disk = main.load_data(args.data_path)
main.validate_data(game_data_disk)
hparams = main.load_hparams(args, game_data_disk)

---------data----------
num_attributes : 3
num_attr_vals : 4
key_support_size : 65
query_support_size : 4096
train_datapoints length : 8388608
val_datapoints length : 4096
sparsity_estimate : 0.17980795755722673
-----------------------
----------hparams----------
seed : 42
batch_size : 128
max_epochs : 1
d_model : 128
embed_dropout : 0.0
model : contrastive
embedding_by_property : False
encoder : lookup
decoder : lookup
d_ff : 1024
num_heads : 1
N_enc : 2
N_dec : 2
attn_wt_tying_scheme : untie_QKVO_zero_O
attn_wt_dropout : 0.0
heads_dropout : 0.0
pff_dropout : 0.0
representation_pos : 0
dotproduct_bottleneck : True
loss_temperature_const : 1.0
loss_smoothing_const : 0.1
nonlinear_classifier_scale_down_factor : [2, 1]
adam_lr : 0.001
adam_beta1 : 0.9
adam_beta2 : 0.999
adam_epsilon : 1e-08
adam_weight_decay : 0
scheduled_adam_beta1 : 0.9
scheduled_adam_beta2 : 0.98
scheduled_adam_epsilon : 1e-09
scheduled_adam_warmup_steps : 4000
gradient_clip_val : 0
debug : False
key_support_size : 65

In [5]:
gt = main.gen_full_matrix(hparams)
x, y = np.nonzero(gt['count_table'])
assert x.shape == y.shape

game_data_true = game_data_disk.copy()
true_points = [(int(ypt), int(xpt)) for xpt, ypt in zip(x, y)]
random.shuffle(true_points)
game_data_true['train_datapoints'] = true_points
game_data_true['val_datapoints'] = true_points[:1024]

Generating Full Matrix
Generated 65 unqiue cards
Generated 4096 cardpairs
-- 2368 cardpairs with shared concept
-- 1728 cardpairs without shared concept
Number of cardpairs per shared concept [(1, 1728), (0, 1728), (2, 576), (3, 64)]
Total number of matches =  47872
Number of matches per key concept hit =  [(1, 43200), (2, 2880), (0, 1728), (3, 64)]
{'shape': (65, 4096), 'size': 266240, 'sparsity': 0.1798076923076923, 'xy_rank': 65, 'xy_div_xyind_rank': 65}


In [6]:
save_path = '../Raw_Datasets/3Attr-4Vals-True.json'
with open(save_path, 'w') as f:
    json.dump(game_data_true, f)

In [7]:
len(game_data_true['train_datapoints'])

47872

In [8]:
import torch

In [13]:
a = torch.tensor([1,6,3])
b = torch.tensor([[1,2,3], [1,2,6]])
b - a.unsqueeze(0)

tensor([[ 0, -4,  0],
        [ 0, -4,  3]])

In [14]:
a - b

tensor([[ 0,  4,  0],
        [ 0,  4, -3]])

In [15]:
(torch.sum((a.unsqueeze(0) - b)**2))**(1/2)

tensor(6.4031)

In [17]:
torch.sum((a.unsqueeze(0) - b)**2)

tensor(41)

In [22]:
(torch.sum((a.unsqueeze(0) - b)**2, dim=-1))**(1/2)

tensor([4., 5.])

In [21]:
torch.sum((a.unsqueeze(0) - b)**2, dim=-1)**(1/2)

tensor([4., 5.])